Import dependencies

In [ ]:
import subprocess
import json
from pprint import pprint

Local hg repo path

In [ ]:
REPO_PATH = "../../autoland"

Get the commits that reference a bug number

In [ ]:
BUG_ID = "Bug 1929024"

hg_command = [
    "hg", "log",
    "-R", REPO_PATH,
    "--template", "###COMMIT###\n{node|short}||{desc}||{date|isodate}",
    "--keyword", BUG_ID
]

try:
    result = subprocess.run(hg_command, capture_output=True, text=True, check=True)
    
    commits = []
    raw_output = result.stdout
    blocks = raw_output.strip().split("###COMMIT###")
    
    for block in blocks:
        if block.strip():
            lines = block.strip().split("||")
            node = lines[0]
            desc = lines[1].strip() if len(lines) > 2 else ""
            date = lines[2]
            commits.append({
                "node": node,
                "desc": desc,
                "date": date
            })

    for commit in commits:
        print(f"Node: {commit['node']}")
        print(f"Desc:\n{commit['desc']}")
        print(f"\nDate: {commit['date']}")
        print("-" * 80)

except subprocess.CalledProcessError as e:
    print(f"Error running Mercurial command: {e}")


- Get all the diffs from autoland hg for the last year
- Each diff should include node/commit hash, descriptions/commit message, code changes
- For each bug id from regressed_by.csv, search the commit messages for that bug id
    -  If there are more than one commits that reference the bug id, gather those commits
        - Only include commits that have "bug 123456" at the beginning of their message or they contain the words "Backed out"
        - combine the effects of those commits (using hg cli) and put them in a new artificial commit
- Add a new column to the bugzilla.csv file and call it diff
- Create a new csv file that has code diffs and call it mercurial.csv